# Mapping COVID19 data in Python

## Help
- keyboard shortcuts
https://yoursdata.net/jupyter-lab-shortcut-and-magic-functions-tips/
- About this interface https://jupyterlab.readthedocs.io/en/stable/user/interface.html
- Plotly Express documentation https://plotly.com/python/plotly-express/
- working with csv and pandas https://towardsdatascience.com/data-science-with-python-intro-to-loading-and-subsetting-data-with-pandas-9f26895ddd7f

## Libraries
Make sure to install libraries

In [ ]:
!pip install plotly
!pip install pandas

Import libraries

In [2]:
import plotly.express as px
import pandas as pd

## Data
Get the data from LA Times
- reference for read_csv https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [3]:
lat = pd.read_csv(
    "https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/latimes-place-totals.csv"
)

Output first 5 rows

In [4]:
lat.head()

,date,county,fips,place,confirmed_cases,note,x,y
0,2020-04-08,Alameda,1,Alameda,18.0,NaN,-122.274444,37.756111
1,2020-04-08,Alameda,1,Albany,NaN,NaN,-122.297778,37.886944
2,2020-04-08,Alameda,1,Berkeley,30.0,NaN,-122.272778,37.871667
3,2020-04-08,Alameda,1,Dublin,14.0,NaN,-121.935833,37.702222
4,2020-04-08,Alameda,1,Emeryville,NaN,NaN,-122.285278,37.831389


How many rows and columns?

In [5]:
lat.shape

(17103, 8)

In [6]:
lat.columns

Index(['date', 'county', 'fips', 'place', 'confirmed_cases', 'note', 'x', 'y'], dtype='object')

Filter out the following:
- empty values (NaN's) 
- incorrect coordinates (ie, positive longitudes)
- null dates

In [10]:
lat = lat.query("confirmed_cases != 'NaN' & x < 0 & date.notnull()", engine='python')  

In [11]:
lat.head()

,date,county,fips,place,confirmed_cases,note,x,y
0,2020-04-08,Alameda,1,Alameda,18.0,NaN,-122.274444,37.756111
2,2020-04-08,Alameda,1,Berkeley,30.0,NaN,-122.272778,37.871667
3,2020-04-08,Alameda,1,Dublin,14.0,NaN,-121.935833,37.702222
5,2020-04-08,Alameda,1,Fremont,40.0,NaN,-121.988611,37.548333
6,2020-04-08,Alameda,1,Hayward,103.0,NaN,-122.080796,37.668820


Order it by date

In [12]:
lat = lat.sort_values(by=["date"], ascending=True)

Output latest entries

In [ ]:
lat.tail()

What is the most current date?

In [ ]:
lastdate = lat.iloc[-1:]['date'].to_list()[0]
lastdate

Filter by `lastdate`

In [ ]:
lat_single_day=lat.loc[lat.date==lastdate]
lat_single_day.head()

Create another filter for just Los Angeles

In [ ]:
lat_LA=lat.loc[lat.county=='Los Angeles']
lat_LA.head()

Now we have three variables to work with:
- `lat`: the entire database
- `lat_single_day`: filtered for one day
- `lat_LA`: just Los Angeles County data

## Stats
Get some stats

In [ ]:
lat.confirmed_cases.describe()

Stats by county

In [ ]:
lat.groupby("county").confirmed_cases.describe().sort_values(by=["max"], ascending=False)

Stats by date

In [ ]:
lat.groupby("date").confirmed_cases.describe()

## Scatter Plots

Create a non-spatial scatter plot, colored by county with date on the x axis

In [ ]:
px.scatter(lat,color="county",x="date",y="confirmed_cases")

Now plot a single day with lat/lon's on the axis. What does it look like?

In [ ]:
px.scatter(lat_single_day,x='x',y='y')

Let's add some color. Color code by confirmed cases

In [ ]:
px.scatter(lat_single_day,x='x',y='y',color='confirmed_cases')

Size it by confirmed cases

In [ ]:
px.scatter(lat_single_day,x='x',y='y',
           color='confirmed_cases', 
           size='confirmed_cases',
           size_max=40, 
           hover_name='place')

Let's try to animate it by date

In [ ]:
px.scatter(lat_LA,x='x',y='y',
           color='confirmed_cases', 
           size='confirmed_cases',
           size_max=60, 
           hover_name='place',
           animation_frame="date",
           animation_group='place')

## Mapbox

In [ ]:
access_token = 'pk.eyJ1IjoieW9obWFuIiwiYSI6IkxuRThfNFkifQ.u2xRJMiChx914U7mOZMiZw'
px.set_mapbox_access_token(access_token)
px.scatter_mapbox(lat_LA, 
                  lat="y", lon="x",     
                  color="confirmed_cases", 
                  size="confirmed_cases",
#                   color_continuous_scale=px.colors.sequential.OrRd, 
                  size_max=30, 
                  opacity=0.5,
                  zoom=6,
                  mapbox_style="dark",
                  hover_name='place',
                  animation_frame="date",
                  animation_group="place",
                 height=600)